# Team information

|Team-number :| 1|
|:----:|:----:|


|Name|    E-Mail        |matriculation-nr.|
|:----:|:----:|:----:|
|Tamara Scherer| schere21@ads.uni-passau.de|104218|
|Felix Müller| muell518@ads.uni-passau.de|104227|


# Import

In [27]:
#from label_studio_sdk import Client
#from label_studio_sdk import project
#from label_studio_sdk import project
#import pandas as pd
#LABEL_STUDIO_URL = 'http://132.231.59.226:8080' #this address needs to be the same as the address the label-studio is hosted on.
#API_KEY = '1655a8922f821195356a17a3224c0532b091c61d' #please add your personal API_Key here to get your API_Key follow the Pictures below

#ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)
#ls.check_connection()
#pro = project.Project.get_from_id(ls,"1")
#tasks = project.Project.get_labeled_tasks(pro)
#tasks[0]

In [28]:
# import labeled data
import pickle
with open("Files/tasks3", 'rb') as f:
    tasks = pickle.load(f)

In [29]:
# import necessary packages
import re
import pandas as pd

In [30]:
# filter the necessary texts with the corresponding label
df = pd.DataFrame(columns = ['Text', 'Label',])

for i in range(len(tasks)): 
    for j in range(len(tasks[i]['annotations'][0]['result'])): 
        df = df.append({
            'Text' : tasks[i]['annotations'][0]['result'][j]['value']['text'],
            'Label' : tasks[i]['annotations'][0]['result'][j]['value']['labels']
                        }, ignore_index = True)
df

,Text,Label
0,Good afternoon and thanks a lot for taking my ...,[QID_1]
1,Good afternoon and thanks a lot for taking my...,[Question_1_Company_specific]
2,Good afternoon and thanks a lot for taking my ...,[Question_2_specific]
3,Good afternoon and thanks a lot for taking my ...,[Question_3_neutral]
4,"No, I think that as it relates to the one, I t...",[AID_1]
...,...,...
11289,One of the areas obviously which has been a st...,[Question_3_neutral]
11290,"Well, let's start with the first part of your ...",[AID_1]
11291,"Well, let's start with the first part of your ...",[Answer_1_specific]
11292,"Well, let's start with the first part of your ...",[Answer_2_positive]


In [31]:
# assign a unique number to each label
def categorise(row):  
    if row['Label'] == ['Question_1_Company_specific']:
        return '1'
    elif row['Label'] == ['Question_1_Market_related']:
        return '2'
    elif row['Label'] == ['Question_2_specific']:
        return '3'
    elif row['Label'] == ['Question_2_open']:
        return '4'
    elif row['Label'] == ['Question_3_attack']:
        return '5'
    elif row['Label'] == ['Question_3_support']:
        return '6'
    elif row['Label'] == ['Question_3_neutral']:
        return '7'
    elif row['Label'] == ['Answer_1_specific']:
        return '8'
    elif row['Label'] == ['Answer_1_avoid_excuse']:
        return '9'
    elif row['Label'] == ['Answer_2_positive']:
        return '10'
    elif row['Label'] == ['Answer_2_negative']:
        return '11'
    elif row['Label'] == ['Answer_3_blame']:
        return '12'
    elif row['Label'] == ['Answer_3_no_blame']:
        return '13'
    else:
        return '0'
    
    
# call function and write results in a new column of the dataframe
df['LabelNumber'] = df.apply(lambda row: categorise(row), axis=1)

df

,Text,Label,LabelNumber
0,Good afternoon and thanks a lot for taking my ...,[QID_1],0
1,Good afternoon and thanks a lot for taking my...,[Question_1_Company_specific],1
2,Good afternoon and thanks a lot for taking my ...,[Question_2_specific],3
3,Good afternoon and thanks a lot for taking my ...,[Question_3_neutral],7
4,"No, I think that as it relates to the one, I t...",[AID_1],0
...,...,...,...
11289,One of the areas obviously which has been a st...,[Question_3_neutral],7
11290,"Well, let's start with the first part of your ...",[AID_1],0
11291,"Well, let's start with the first part of your ...",[Answer_1_specific],8
11292,"Well, let's start with the first part of your ...",[Answer_2_positive],10


**Filter one ID stage (e.g. Question_1_XX)**

In [32]:
idStage = ['1', '2']
df = df[df['LabelNumber'].isin(idStage)]

# set new index
df = df.reset_index()
# delete old indices
df.pop('index')
df

,Text,Label,LabelNumber
0,Good afternoon and thanks a lot for taking my...,[Question_1_Company_specific],1
1,"Okay, that's very helpful. And then on your gr...",[Question_1_Company_specific],1
2,"Hi Richard. So, on gross margin, it looked pre...",[Question_1_Market_related],2
3,And the core-on-core was pretty normal for you...,[Question_1_Company_specific],1
4,Okay. And my follow-up is on the EBIT dollar ...,[Question_1_Company_specific],1
...,...,...,...
1304,Hi. This is Mark for Pat. Thank you so much fo...,[Question_1_Company_specific],1
1305,Yes. I'm just wondering if you could talk a li...,[Question_1_Company_specific],1
1306,"Hi, thank you. Congrats on the quarter. Just a...",[Question_1_Company_specific],1
1307,Thanks for squeezing me in guys. This question...,[Question_1_Company_specific],1


# Preprocessing

In [33]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\ts23\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


In [34]:
# necessary imports for preprocessing steps

import string
import nltk

# import a stemmer for english words
snowStem = nltk.stem.SnowballStemmer('english')

# import stopwords
from nltk.corpus import stopwords
en_stopwords = stopwords.words('english')

# import for tokenization
from nltk import word_tokenize
nltk.download('punkt')

# import for lemmatizer
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,pos_tag
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ts23\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ts23\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ts23\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ts23\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [35]:
# function that includes all preprocessing steps

def do_preprocessing(text_to_clean, remove_stopwords = True, stemming = True):
    # remove numbers 
    text_to_clean = re.sub(r'\d+', '', text_to_clean)
    # remove leading and ending white spaces
    text_to_clean = text_to_clean.strip()
    # transform text to lower case
    text_to_clean = text_to_clean.lower()

    if stemming:
      # stemming
      text_to_clean = snowStem.stem(text_to_clean)
    
    if remove_stopwords:
        # remove stop words
        text_to_clean = ' '.join([w for w in text_to_clean.split() if not(w in en_stopwords)])

    # remove punctuation
    text_to_clean = text_to_clean.translate(str.maketrans('','', string.punctuation))
    
    return text_to_clean


# call function and write results in new column of the dataframe
df.loc[:, 'CleanText'] = df['Text'].apply(
    lambda x: do_preprocessing(x, True, True))

df

,Text,Label,LabelNumber,CleanText
0,Good afternoon and thanks a lot for taking my...,[Question_1_Company_specific],1,good afternoon thanks lot taking question so r...
1,"Okay, that's very helpful. And then on your gr...",[Question_1_Company_specific],1,okay thats helpful growth china surpass expect...
2,"Hi Richard. So, on gross margin, it looked pre...",[Question_1_Market_related],2,hi richard so gross margin looked pretty solid...
3,And the core-on-core was pretty normal for you...,[Question_1_Company_specific],1,coreoncore pretty normal well
4,Okay. And my follow-up is on the EBIT dollar ...,[Question_1_Company_specific],1,okay followup ebit dollar growth looked like c...
...,...,...,...,...
1304,Hi. This is Mark for Pat. Thank you so much fo...,[Question_1_Company_specific],1,hi mark pat thank much taking question could t...
1305,Yes. I'm just wondering if you could talk a li...,[Question_1_Company_specific],1,yes im wondering could talk little bit custome...
1306,"Hi, thank you. Congrats on the quarter. Just a...",[Question_1_Company_specific],1,hi thank you congrats quarter followup toms qu...
1307,Thanks for squeezing me in guys. This question...,[Question_1_Company_specific],1,thanks squeezing guys question you john know g...


In [36]:
# function for lemmatization
def lemmatization(text):
    
  text = word_tokenize(text)
  
  result=[]
  wordnet = WordNetLemmatizer()
  for token,tag in pos_tag(text):
        pos=tag[0].lower()
        
        if pos not in ['a', 'r', 'n', 'v']:
            pos='n'
            
        result.append(wordnet.lemmatize(token,pos))

  return result

# call function
df['CleanText'] = df['CleanText'].apply(
    lambda x: lemmatization(x))

df

,Text,Label,LabelNumber,CleanText
0,Good afternoon and thanks a lot for taking my...,[Question_1_Company_specific],1,"[good, afternoon, thanks, lot, take, question,..."
1,"Okay, that's very helpful. And then on your gr...",[Question_1_Company_specific],1,"[okay, thats, helpful, growth, china, surpass,..."
2,"Hi Richard. So, on gross margin, it looked pre...",[Question_1_Market_related],2,"[hi, richard, so, gross, margin, look, pretty,..."
3,And the core-on-core was pretty normal for you...,[Question_1_Company_specific],1,"[coreoncore, pretty, normal, well]"
4,Okay. And my follow-up is on the EBIT dollar ...,[Question_1_Company_specific],1,"[okay, followup, ebit, dollar, growth, look, l..."
...,...,...,...,...
1304,Hi. This is Mark for Pat. Thank you so much fo...,[Question_1_Company_specific],1,"[hi, mark, pat, thank, much, take, question, c..."
1305,Yes. I'm just wondering if you could talk a li...,[Question_1_Company_specific],1,"[yes, im, wondering, could, talk, little, bit,..."
1306,"Hi, thank you. Congrats on the quarter. Just a...",[Question_1_Company_specific],1,"[hi, thank, you, congrats, quarter, followup, ..."
1307,Thanks for squeezing me in guys. This question...,[Question_1_Company_specific],1,"[thanks, squeeze, guy, question, you, john, kn..."


**Create dictionary**

In [37]:
from gensim import corpora

# generate the gensim dictionary
dct = corpora.dictionary.Dictionary(df['CleanText']).values()

In [38]:
# convert tokens back to strings for further processing
string1=" "
df['CleanText'] = df['CleanText'].apply(
    lambda x: string1.join(x))

df

,Text,Label,LabelNumber,CleanText
0,Good afternoon and thanks a lot for taking my...,[Question_1_Company_specific],1,good afternoon thanks lot take question so rec...
1,"Okay, that's very helpful. And then on your gr...",[Question_1_Company_specific],1,okay thats helpful growth china surpass expect...
2,"Hi Richard. So, on gross margin, it looked pre...",[Question_1_Market_related],2,hi richard so gross margin look pretty solid w...
3,And the core-on-core was pretty normal for you...,[Question_1_Company_specific],1,coreoncore pretty normal well
4,Okay. And my follow-up is on the EBIT dollar ...,[Question_1_Company_specific],1,okay followup ebit dollar growth look like com...
...,...,...,...,...
1304,Hi. This is Mark for Pat. Thank you so much fo...,[Question_1_Company_specific],1,hi mark pat thank much take question could tal...
1305,Yes. I'm just wondering if you could talk a li...,[Question_1_Company_specific],1,yes im wondering could talk little bit custome...
1306,"Hi, thank you. Congrats on the quarter. Just a...",[Question_1_Company_specific],1,hi thank you congrats quarter followup tom que...
1307,Thanks for squeezing me in guys. This question...,[Question_1_Company_specific],1,thanks squeeze guy question you john know guy ...


# Split of training and test dataset

In [39]:
import numpy as np

training_fraction = 0.70
training_size = int(np.floor(len(df) * training_fraction))

data_train, data_test = df[:training_size], df[training_size:]
#X_train, X_test = data_train.drop(['monthly_stock_return'], axis = 1), data_test.drop(['monthly_stock_return'], axis = 1)
#y_train, y_test = data_train[['monthly_stock_return']], data_test[['monthly_stock_return']]

# Bag-of-Words (BOW)

In [40]:
from sklearn.feature_extraction.text import CountVectorizer

def do_bow(data):
    count_vec = CountVectorizer(vocabulary = dct)

    bow = count_vec.fit_transform(data['CleanText'])
    bow_matrix = pd.DataFrame(data = bow.toarray(), columns = count_vec.get_feature_names_out())

    display(bow_matrix)
    
    return(bow_matrix)

# call function
train_bow_matrix = do_bow(data_train)
test_bow_matrix = do_bow(data_test)

,afternoon,anniversary,benefit,decision,different,do,drive,give,good,growth,...,realignment,rpo,shed,asics,hock,jericho,optical,performing,tomahawk,trident
0,1,1,1,1,1,1,2,1,1,2,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
912,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
913,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
914,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


,afternoon,anniversary,benefit,decision,different,do,drive,give,good,growth,...,realignment,rpo,shed,asics,hock,jericho,optical,performing,tomahawk,trident
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
389,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
390,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
391,0,0,0,0,0,0,0,1,0,0,...,0,2,1,0,0,0,0,0,0,0


# TF-IDF

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer

def do_tfidf(data):
    tfidf_vec = TfidfVectorizer(vocabulary = dct)

    tfidf = tfidf_vec.fit_transform(data['CleanText'])
    tfidf_matrix = pd.DataFrame(data = tfidf.toarray(), columns = tfidf_vec.get_feature_names_out())

    display(tfidf_matrix)
    
    return(tfidf_matrix)

# call function
train_tfidf_matrix = do_tfidf(data_train)
test_tfidf_matrix = do_tfidf(data_test)

,afternoon,anniversary,benefit,decision,different,do,drive,give,good,growth,...,realignment,rpo,shed,asics,hock,jericho,optical,performing,tomahawk,trident
0,0.153021,0.236999,0.155793,0.177425,0.126805,0.153021,0.243532,0.085938,0.115618,0.194891,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.174402,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.152685,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
912,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.237623,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
913,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
914,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.093197,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,afternoon,anniversary,benefit,decision,different,do,drive,give,good,growth,...,realignment,rpo,shed,asics,hock,jericho,optical,performing,tomahawk,trident
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.0,0.0,0.0,0.145685,0.0,0.0,0.087808,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
389,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
390,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.180413,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
391,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.093644,0.0,0.000000,...,0.000000,0.464447,0.232224,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# LSA

In [42]:
from scipy.sparse.linalg import svds

def do_lsa(data):
    num_components = 10
    q, s, p = svds(data, k = num_components)

    lsa_doc_matrix = pd.DataFrame(data=q)

    display(lsa_doc_matrix)

    return(lsa_doc_matrix)

# call function
train_lsa_matrix = do_lsa(train_tfidf_matrix)
test_lsa_matrix = do_lsa(test_tfidf_matrix)

,0,1,2,3,4,5,6,7,8,9
0,-0.063852,0.001987,-0.017864,0.101711,-0.045427,0.036281,0.051261,0.006285,-0.017381,0.044309
1,-0.047987,0.020022,-0.008672,0.021375,0.035792,0.001680,0.039789,0.006914,-0.015449,0.018086
2,-0.026689,0.022862,-0.013200,0.021622,-0.027197,-0.011017,-0.065299,0.003146,0.033844,0.049235
3,0.003002,-0.003607,0.005458,0.019330,-0.013714,-0.007566,-0.002486,-0.010854,0.011877,0.008377
4,-0.076328,0.032857,-0.035851,0.011290,0.007317,-0.074875,-0.005319,0.090881,0.057036,0.062475
...,...,...,...,...,...,...,...,...,...,...
911,-0.012822,0.009924,-0.009228,-0.045003,0.012187,0.011660,-0.042534,0.017822,-0.017820,0.038597
912,0.016243,0.016886,0.003163,-0.036650,0.014578,0.031850,-0.014996,-0.002250,-0.003314,0.022267
913,-0.004968,0.031451,-0.037696,-0.015868,-0.002805,-0.008951,-0.007917,0.017709,-0.000577,0.017045
914,0.011292,0.023081,-0.005894,-0.018354,0.007153,0.027412,-0.009124,0.012126,0.001633,0.014872


,0,1,2,3,4,5,6,7,8,9
0,0.003787,-0.036151,-0.007546,0.012739,0.003463,0.007904,-0.004376,0.007969,-0.003698,-0.017286
1,0.003623,0.010465,-0.016374,-0.023581,-0.063199,0.061785,-0.001506,-0.028464,0.013361,-0.032609
2,-0.100248,-0.064394,0.033927,-0.012630,-0.018216,0.003165,-0.027062,-0.008933,-0.002267,-0.041443
3,0.048395,-0.064332,0.107475,-0.191095,-0.091670,0.121379,0.023276,0.035205,-0.030819,-0.048567
4,0.056217,-0.127080,0.056279,-0.160083,-0.100818,0.076546,-0.001372,0.058783,0.028786,-0.032502
...,...,...,...,...,...,...,...,...,...,...
388,0.049372,0.106876,-0.133016,0.001300,0.119362,0.064732,0.067216,0.153516,0.105204,-0.070174
389,0.082668,0.025952,-0.129962,0.023329,0.034594,-0.000819,0.077375,0.149159,0.101139,-0.048324
390,-0.077289,0.033382,0.031542,0.057291,-0.020551,0.019505,0.065074,-0.020076,0.052925,-0.054011
391,-0.000596,0.030342,-0.037173,0.004155,0.058917,-0.033699,0.011284,0.106086,0.040183,-0.072156


# Doc2Vec

In [47]:
import gensim
from gensim.models.doc2vec import Doc2Vec

def do_doc2vec(data):
    tagged_documents = []
    sentences = [text.split() for text in data['CleanText']]

    for i, doc in enumerate(sentences):
        tagged_documents.append(gensim.models.doc2vec.TaggedDocument(doc, [i]))

    d2v = Doc2Vec(vector_size=50, min_count=2, epochs=40)
    d2v.build_vocab(tagged_documents)

    d2v.train(tagged_documents, total_examples=d2v.corpus_count, epochs=d2v.epochs)

    doc2vec_alldocs = []
    for i in range(len(tagged_documents)):
        doc2vec_alldocs.append(d2v.infer_vector(tagged_documents[i].words))

    doc2vec_alldocs_matrix = pd.DataFrame(doc2vec_alldocs)

    display(doc2vec_alldocs_matrix)
    
    return(doc2vec_alldocs_matrix)

# call function
train_doc2vec_matrix = do_doc2vec(data_train) 
test_doc2vec_matrix = do_doc2vec(data_test)

           0         1         2         3         4         5         6   \
0    0.272539 -0.300007 -0.105069 -0.065546 -0.497454 -0.130558  0.157052   
1    0.371789  0.326099  0.100768 -0.281835 -0.122604  0.068374  0.356012   
2   -0.210710 -0.069187 -0.272739  0.724354 -0.050572 -0.950133 -0.244594   
3   -0.090125 -0.108816  0.136998  0.010027 -0.084013 -0.205099  0.216724   
4   -0.662580  0.252180  0.210383 -0.652609 -0.152201 -1.124586 -0.301666   
..        ...       ...       ...       ...       ...       ...       ...   
911 -0.081770 -0.262791 -0.107932 -0.098519  0.262490 -0.107922  0.193070   
912  0.161228 -0.136499 -0.341065 -0.196237 -0.307080  0.149202  0.260762   
913 -0.256742 -0.097598 -0.039107  0.135796  0.157405 -0.279051 -0.058311   
914  0.237960 -0.181770  0.307379 -0.166051 -0.140563 -0.041127  0.295432   
915 -0.380930 -0.650418  0.411126 -0.110676 -0.119092 -0.303857  0.378769   

           7         8         9   ...        40        41        42  \
0  

# Generating a Neural Network

In [45]:
import tensorflow as tf

# first we define the network, units is the number of neurons, in the first layer we need to tell the model the input shape
neural_network = tf.keras.Sequential([
                    # hidden layer
                    tf.keras.layers.Dense(units = 100, input_shape = [data_train.shape[1]], activation = 'sigmoid'),
                    # output layer - as we want probability predictions, it is important to use the sigmoid activation
                    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

# by compilation we define the loss function which is supposed to be minimized and which optimization method should be used
neural_network.compile(loss = 'binary_crossentropy', optimizer = 'sgd',  metrics = ['accuracy', tf.keras.metrics.Recall()])

# a summary for all parameters which need to be estimated
neural_network.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               500       
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 601
Trainable params: 601
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# we fit the mode, epochs is the number of steps which are repeated using gradient descent
history = neural_network.fit(train, data_train, epochs = 100, validation_data = (test, data_test))